vamos a empezar de nuevo con lo del sop para ver si sale

In [ ]:
import numpy as np
import random 
import csv
from operator import itemgetter

In [ ]:
#funcion para obtener el costo

def obtenerCosto (solucion, costos, reglas):
  #obtener el total de nodos -1 (no es necesario revisar el ultimo nodo)
  nodos = len(solucion) -1

  #se obtine el costo máximo de la tabla
  costoMax = max (max(fila) for fila in costos)
  costoTotal = 0

  #se suman los costos C de la solución
  for i in range(nodos):
    costoTotal += costos[solucion[i]][solucion[i+1]]
  n =  presedencia(solucion, reglas)
  costoTotal += n* costoMax  
  return costoTotal

In [ ]:
def presedencia(solucion, rules):
  #tiene que variar estos arreglos dependiendo de la cantidad de reglas
  auxB = np.zeros(len(rules))
  cont = 0

  for i in range (len(solucion)):
    for j in range(len(rules)):
      if rules[j][0] == solucion[i]:
        auxB[j]+=1
      if auxB[j]== 1:
        if rules[j][1] == solucion[i]:
          auxB[j]+=1
  for i in range(len(auxB)):
    if auxB[i] <2:
      cont +=1
  return cont

In [ ]:
def solucionInicial(numNodos,reglas):
    solucionTemp = []
    solucion = []
    #generacion de numeros aleatorios con su indice para obtener la primera solucion
    for i in range(numNodos-2):
        solucionTemp.append([i+1,random.random()])
    #Se reordenan para poder decidir en que orden visitar los nodos
    solucionTemp.sort(key=itemgetter(1))
    # El primero nodo debe ser siempre el cero
    solucion.append(0)
    #agregar los demas nodos
    for i in list(solucionTemp):
        solucion.append(i[0])
    #El ultimo nodo debe ser numNodos-1
    solucion.append(numNodos-1)
    solucion = corregirPrecedencia(solucion[:],reglas,numNodos)
    return solucion

In [ ]:
def solucionVecino(solucion,reglas):
    #generar las posiciones a cambiar
    pos1 = random.randint(1,len(solucion)-2)
    pos2 = random.randint(1,len(solucion)-2)
    while (pos1 == pos2):
        pos2 = random.randint(1,len(solucion)-2)
    #print(pos1, ',', pos2)
    #Relizamos el cambio de acuerdo a los indices 
    solucion[pos1],solucion[pos2] = solucion[pos2],solucion[pos1]
    solucion = corregirPrecedencia(solucion[:],reglas,len(solucion))
    return solucion

In [ ]:
def corregirPrecedencia(solucion, reglas, numNodos):
    # Se recorre la lista de reglas
    for regla in reglas:
        nodoA = regla[0]
        nodoB = regla[1]

        # Se busca la posición de los nodos en la solución actual
        posA = solucion.index(nodoA)
        posB = solucion.index(nodoB)

        # Si la posición de B es menor que la de A, se intercambian los nodos
        if posB < posA:
            solucion[posA], solucion[posB] = solucion[posB], solucion[posA]

    # Se verifica que la solución corregida contenga todos los nodos
    assert set(solucion) == set(range(numNodos))

    return solucion


In [ ]:
def readFile(size, name):
    m = np.zeros((0), dtype=int)
    m_prec = np.zeros((0), dtype=int)
    
    with open(name, newline='') as File:  
        reader = csv.reader(File, delimiter=',', quotechar=',',quoting=csv.QUOTE_MINIMAL)
        i=0
        for row in File:
            
            row = row.rstrip()
            separador = ","
            row = row.split(",")
            row = list(map(int, row))
            m_np = np.array(row)
            
            if i < size :
                m = np.append(m, m_np, axis=0)
                #print(m_np)
            else:
                m_prec = np.append(m_prec, m_np, axis=0)
            i+=1
    
    m = np.array(m).reshape(size,size)
    m_prec = np.array(m_prec).reshape(int((len(m_prec)/2)),2)
    return m, m_prec

In [ ]:
name = '100.txt'
nodos = 100
m, m_prec = readFile(nodos, name)

print(f'{m}')


[[19  8 13 ...  2  2  4]
 [ 2  6  3 ... 10  8  6]
 [ 3 14 10 ... 20 13 11]
 ...
 [13 10 14 ... 18 19 20]
 [ 3 13 16 ...  0  0 18]
 [ 8  1  0 ...  4  0 12]]


In [ ]:
print(m_prec)

[[ 1  3]
 [ 1  4]
 [ 1 13]
 [ 8  7]
 [15 22]
 [48 49]]


Una implementación con busqueda exhaustiva

In [ ]:
a = solucionInicial(100, m_prec)
print (obtenerCosto(a, m, m_prec))
c = solucionVecino(a,m_prec)
print(c)

994
[0, 17, 79, 29, 86, 12, 93, 84, 20, 56, 2, 26, 8, 68, 41, 28, 18, 91, 15, 95, 14, 71, 52, 1, 72, 30, 16, 38, 59, 89, 69, 90, 31, 64, 55, 66, 54, 39, 32, 40, 7, 74, 57, 24, 13, 37, 10, 76, 61, 67, 53, 34, 97, 80, 65, 25, 70, 82, 94, 35, 11, 88, 3, 96, 83, 60, 43, 45, 44, 50, 33, 9, 36, 58, 22, 62, 81, 23, 47, 75, 6, 77, 92, 4, 42, 63, 21, 48, 87, 85, 78, 19, 46, 98, 73, 51, 49, 27, 5, 99]


In [ ]:
def local_search(cost_matrix, reglas, numNodos, max_iterations):
    # Algoritmo de búsqueda local para el SO con matriz de costos
    
    best_seq = solucionInicial(numNodos, reglas)
    best_cost = obtenerCosto(best_seq, cost_matrix, reglas )
    for i in range(max_iterations):
        # Intercambia  nodos aleatorios en la secuencia
        new_sol = solucionVecino(best_seq, reglas)
        # Calcula el costo de la nueva secuencia
        cost = obtenerCosto(new_sol,cost_matrix,reglas)
        # Si la nueva secuencia es mejor, actualiza la mejor solución
        if cost < best_cost:
            best_seq = new_sol.copy()
            best_cost = cost
    return best_seq, best_cost

In [ ]:
best_seq, best_cost = local_search(m, m_prec, 100, 10000)

print (f'El mejor costo es {best_cost} con la siguiente secuencia \n{best_seq}')

El mejor costo es 747 con la siguiente secuencia 
[0, 3, 86, 66, 41, 52, 31, 32, 97, 91, 49, 77, 69, 8, 68, 29, 6, 57, 17, 34, 15, 70, 44, 33, 18, 12, 92, 10, 1, 19, 51, 37, 13, 80, 26, 74, 87, 48, 2, 20, 96, 55, 93, 90, 42, 89, 72, 79, 61, 7, 21, 16, 22, 4, 58, 36, 40, 56, 84, 78, 76, 85, 50, 82, 14, 95, 73, 28, 75, 83, 94, 9, 81, 27, 5, 60, 59, 23, 47, 53, 65, 45, 67, 30, 43, 46, 63, 38, 39, 64, 88, 24, 71, 62, 35, 11, 98, 25, 54, 99]
